In [6]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
using Random
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_CTMRG_unitcell.jl")
include("kagome_model.jl")
include("kagome_model_cell.jl")
include("kagome_IPESS.jl")
include("kagome_FiniteDiff.jl")



Random.seed!(1234)


D=6;



theta=0.2*pi;
J1=cos(theta);
J2=0;
J3=0;
Jchi=0;
Jtrip=sin(theta);

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);



#state_dict=read_json_state("LS_D_8_chi_40.json")
#init_statenm=nothing;
init_statenm="julia_LS_D_6_chi_40.json"
#init_statenm=nothing
init_noise=0;
CTM_conv_tol=1e-6;
CTM_ite_nums=100;
CTM_trun_tol=1e-12;
Bond_irrep="A";
Triangle_irrep="A1+iA2";
#nonchiral="A1_even";
nonchiral="No"


"No"

In [2]:
state_dict, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe, _, _=initial_state(Bond_irrep,Triangle_irrep,nonchiral,D,init_statenm,init_noise)

load state: julia_LS_D_6_chi_40.json


(Dict{String, Any}("coes" => Dict{String, Dict{String, Any}}("Triangle_A2_coe" => Dict("dtype" => "float64", "numEntries" => 2, "entries" => Any["0 0.7369944794910288", "1 -0.007354087925930567"], "dims" => Any[2]), "Bond_A_coe" => Dict("dtype" => "float64", "numEntries" => 2, "entries" => Any["0 0.9108232898667802", "1 -0.4130940700485079"], "dims" => Any[2]), "Triangle_A1_coe" => Dict("dtype" => "float64", "numEntries" => 3, "entries" => Any["0 0.13309520130616825", "1 0.6611270054395537", "2 0.03945748172585299"], "dims" => Any[3])), "nonchiral" => "No", "Bond_irrep" => "A", "Triangle_irrep" => "A1+iA2"), [0.9108232898667802, -0.4130940700485079], Any[], [0.13309520130616825, 0.6611270054395537, 0.03945748172585299], [0.7369944794910288, -0.007354087925930567], [0.0, 0.0, 1.0], [1.0, 0.0])

In [3]:

    A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);
    
    bond_tensor,triangle_tensor=construct_su2_PG_IPESS(state_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);


    PEPS_tensor=bond_tensor;
    @tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
    A_unfused=PEPS_tensor;
    
    U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
    @tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

    CTM=[];
    U_L=[];
    U_D=[];
    U_R=[];
    U_U=[];

    init=Dict([("CTM", []), ("init_type", "PBC")]);
    conv_check="singular_value";
    CTM_ite_info=true;
    CTM_conv_info=true;
    chi=20;
    CTM, AA_fused, U_L,U_D,U_R,U_U,ite_num,ite_err=CTMRG(A_fused,chi,conv_check,CTM_conv_tol,init,CTM_ite_nums,CTM_trun_tol,CTM_ite_info,CTM_conv_info);
    
    E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
    #E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");
    energy=(E_up*2)/3;
    println(energy)

    #return energy,CTM,U_L,U_D,U_R,U_U

    chiral_order_parameters=Dict([("J1", 0), ("J2", 0), ("J3", 0), ("Jchi", 0), ("Jtrip", 1)]);
    chiral_order_up, chiral_order_down=evaluate_ob(chiral_order_parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");





"initialize CTM"

start CTM iterations:
CTMRG iteration: 1, CTMRG err: 4.078634148980675


CTMRG iteration: 2, CTMRG err: 0.4833906073454223


CTMRG iteration: 3, CTMRG err: 0.692925505342094


CTMRG iteration: 4, CTMRG err: 0.018284743957585867


...

CTMRG iteration: 13, CTMRG err: 9.276224659407755e-6


CTMRG iteration: 14, CTMRG err: 4.052689370107061e-6


CTMRG iteration: 15, CTMRG err: 1.7703019126929002e-6


CTMRG iteration: 16, CTMRG err: 7.727957573239436e-7


-0.34929794092622585 + 1.338259025297828e-17im


In [31]:
# Lx=2;Ly=2;
# A_cell=Matrix(undef,Lx,Ly);
# A_cell[1,1]=A_fused;
# A_cell[1,2]=A_fused;
# A_cell[2,1]=A_fused;
# A_cell[2,2]=A_fused;

# init=Dict([("CTM", []), ("init_type", "PBC")]);
# conv_check="singular_value";
# CTM_ite_info=true
# CTM_conv_info=true;
# CTM, AA_fused, U_L,U_D,U_R,U_U,ite_num,ite_err=CTMRG_cell(A_cell,chi,conv_check,CTM_conv_tol,init,CTM_ite_nums,CTM_trun_tol,CTM_ite_info,CTM_conv_info);


In [4]:
id_L=unitary(space(A_fused,1),space(A_fused,1));
id_D=unitary(space(A_fused,2),space(A_fused,2));
id_phy=unitary(space(A_fused,5),space(A_fused,5));


@tensor A_LD[:]:=id_L[-1,-3]*id_D[-4,-2];
@tensor A_RU[:]:=id_L[-5,-1]*id_D[-3,-6]*id_phy[-2,-4];
U_L_phy=unitary(fuse(space(A_RU,1)⊗ space(A_RU,2)), space(A_RU,1)⊗ space(A_RU,2));
U_D_phy=unitary(fuse(space(A_fused,4)⊗ space(A_fused,5)), space(A_fused,4)⊗ space(A_fused,5));
@tensor A_LU[:]:=A_fused'[-1,-2,1,-4,2]*U_L_phy'[1,2,-3];
@tensor A_RD[:]:=A_fused[-1,-2,-3,1,2]*U_D_phy[-4,1,2];
@tensor A_RU[:]:=A_RU[1,2,3,4,-3,-4]*U_D_phy'[3,4,-2]*U_L_phy[-1,1,2];

# @tensor tt[:]:=A_RU[-1,1,-2,-3]*A_RD[-4,-5,-6,1];
# @tensor tt[:]:=A_LU[-1,-2,1,-3]*A_RU[1,-4,-5,-6];

Lx=2;Ly=2;
A_cell=Matrix(undef,Lx,Ly);
A_cell[1,1]=A_LU;
A_cell[2,1]=A_RU;
A_cell[1,2]=A_LD;
A_cell[2,2]=A_RD;


In [7]:
init=Dict([("CTM", []), ("init_type", "single_layer_random")]);
conv_check="singular_value";
CTM_ite_info=true
CTM_conv_info=true;
chi=20;
CTM_chi_20, _, _,_,_,_=CTMRG_cell(A_cell,chi,conv_check,CTM_conv_tol,init,CTM_ite_nums,CTM_trun_tol,CTM_ite_info,CTM_conv_info);


"initialize CTM"

18×18 Matrix{Float64}:
 0.695527  0.0       0.0       0.0        …  0.0        0.0        0.0
 0.0       0.390188  0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.110673  0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0496653     0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0        …  0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0        …  0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0      

start CTM iterations:
TensorMap(ProductSpace(Rep[SU₂](0=>10, 1/2=>15, 1=>10, 3/2=>5)) ← ProductSpace(Rep[SU₂](0=>10, 1/2=>15, 1=>10, 3/2=>5))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 26.82726905866821   0.0                0.0                0.0                 0.0                 0.0                 0.0                 0.0                 0.0                   0.0
  0.0               15.049995781733344  0.0                0.0                 0.0                 0.0                 0.0                 0.0                 0.0                   0.0
  0.0                0.0                4.268772430121364  0.0                 0.0                 0.0                 0.0                 0.0                 0.0                   0.0
  0.0                0.0                0.0                1.9156490575592124  0.0                 0.0                 0.0                 0.0                 0.0                   0.0

20×20 Matrix{Float64}:
 0.753564  0.0       0.0       0.0        …  0.0        0.0        0.0
 0.0       0.321881  0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.032313  0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0234497     0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0        …  0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0        …  0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0        0.0        0.0
 0.0       0.0       0.0       0.0           0.0      

  0.0                 0.0                 0.0                 0.0                  0.0                    0.0
  0.0                 0.0                0.0                 0.0                 0.8925134038849769  0.0                 0.0                 0.0                  0.0                    0.0
  0.0                 0.0                0.0                 0.0                 0.0                 0.2081755267501035  0.0                 0.0                  0.0                    0.0
  0.0                 0.0                0.0                 0.0                 0.0                 0.0                 0.0561107800706591  0.0                  0.0                    0.0
  0.0                 0.0                0.0                 0.0                 0.0                 0.0                 0.0                 0.00613317978364787  0.0                    0.0
  0.0                 0.0                0.0                 0.0                 0.0                 0.0                 0.0          

...

20×20 Matrix{Float64}:
 0.804731  0.0         0.0         …  0.0         0.0         0.0
 0.0       0.00110213  0.0            0.0         0.0         0.0
 0.0       0.0         5.14313e-5     0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0         …  0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0         …  0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0         0.0
 0.0       0.0         0.0            0.0         0.0

18×18 Matrix{Float64}:
 0.80768  0.0         0.0          0.0       …  0.0          0.0
 0.0      0.00139548  0.0          0.0          0.0          0.0
 0.0      0.0         0.000140649  0.0          0.0          0.0
 0.0      0.0         0.0          0.416615     0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0       …  0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0       …  0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0.0      0.0         0.0          0.0          0.0          0.0
 0

19×19 Matrix{Float64}:
 0.999086  0.0          0.0         …  0.0         0.0         0.0
 0.0       0.000393529  0.0            0.0         0.0         0.0
 0.0       0.0          9.59653e-6     0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0         …  0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0         …  0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0            0.0         0.0         0.0
 0.0       0.0          0.0            

18×18 Matrix{Float64}:
 0.806462  0.0          0.0        …  0.0         0.0         0.0
 0.0       0.000951733  0.0           0.0         0.0         0.0
 0.0       0.0          4.6782e-5     0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0        …  0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0        …  0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0

19×19 Matrix{Float64}:
 0.999069  0.0          0.0        …  0.0         0.0         0.0
 0.0       0.000380135  0.0           0.0         0.0         0.0
 0.0       0.0          1.0352e-5     0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0        …  0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0        …  0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0         0.0
 0.0       0.0          0.0           0.0         0.0

In [9]:
println(space(CTM["Cset"][1][1,1]))
println(space(CTM["Cset"][1][2,1]))
println(space(CTM["Cset"][1][1,2]))
println(space(CTM["Cset"][1][2,2]))

(Rep[SU₂](0=>2, 1/2=>2, 1=>3, 3/2=>1)' ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>3, 3/2=>1)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()
(Rep[SU₂](0=>3, 1/2=>4, 1=>3)' ⊗ Rep[SU₂](0=>3, 1/2=>4, 1=>3)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()
(Rep[SU₂](0=>3, 1/2=>4, 1=>3)' ⊗ Rep[SU₂](0=>3, 1/2=>4, 1=>3)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()
(Rep[SU₂](0=>3, 1/2=>5, 1=>2)' ⊗ Rep[SU₂](0=>3, 1/2=>5, 1=>2)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()


In [24]:
init=Dict([("CTM", CTM_chi_20), ("init_type", "single_layer_random")]);
conv_check="singular_value";
CTM_ite_info=true
CTM_conv_info=true;
chi=80;
CTM, _, _,_,_,_=CTMRG_cell(A_cell,chi,conv_check,CTM_conv_tol,init,CTM_ite_nums,CTM_trun_tol,CTM_ite_info,CTM_conv_info);


start CTM iterations:
CTMRG iteration: 1, CTMRG err: 8.690712982754198


CTMRG iteration: 2, CTMRG err: 0.11310736597521406


CTMRG iteration: 3, CTMRG err: 0.04885741340914259


CTMRG iteration: 4, CTMRG err: 0.05087327712024688


CTMRG iteration: 5, CTMRG err: 0.05081023089704274


...

CTMRG iteration: 7, CTMRG err: 0.05119337670565935


CTMRG iteration: 8, CTMRG err: 0.051253730166310414
CTMRG iteration: 9, CTMRG err: 0.05123955701778578


CTMRG iteration: 10, CTMRG err: 0.05125420472861292


CTMRG iteration: 11, CTMRG err: 0.05125108492083653


CTMRG iteration: 12, CTMRG err: 0.051255163025152414


In [22]:

method1="E_triangle";
method2="full_cell";
E_up=evaluate_ob_UpTriangle_single_layer(parameters, U_phy, U_D_phy, A_cell, CTM, method1, method2);
energy=(E_up*2)/3;
println(energy)

method2="reduced_cell";
E_up=evaluate_ob_UpTriangle_single_layer(parameters, U_phy, U_D_phy, A_cell, CTM, method1, method2);
energy=(E_up*2)/3;
println(energy)


-0.34929674193266574 + 1.266962877281548e-6im
-0.34929970530601717 - 3.9019848749972145e-6im


-0.17278520614840587 - 2.6015358739631133e-6im-0.17279570396374117 - 5.343584001061292e-7im-0.17278680734224794 + 6.013207482535386e-6im
